# Блок импорта библиотек

In [ ]:
import pandas as pd
import numpy as np
import math
import re

# Блок функций

Функция замены латинских букв на кириллические

In [ ]:
def replace_letters(text):
  # создаем словарь соответствия латинских букв и кириллицы
  mapping = {
      'A': 'А',  # пример: латинская A -> кириллическая А
      'E': 'Е',
      'K': 'К',
      'M': 'М',
      'O': 'О',
      'T': 'Т',
      'C': 'С',
      'P': 'Р',
      'X': 'Х'
      }
  # заменяем каждую букву из текста, если она есть в словаре
  return ''.join(mapping.get(char, char) for char in text)

Функция для поиска автомобильных номеров

In [ ]:
def transport_number(text):
  # проверка на наличие данных
  if pd.notna(text):
    text = str(text)

    # корректировка номеров ТС - удаление пробелов, (, ) и /, приведение к верхнему регистру
    processed_text = re.sub(r'[\s\(\)\/\-\\]', '', str(text)).upper()

    processed_text = replace_letters(processed_text)

    # паттерн для автомобильного номера
    pattern = r'[А-ЯA-Z]\d{3}[А-ЯA-Z]{2}\d{0,3}'

    # ищем совпадение по паттерну, если находим - функция возвращает весь номер
    res = re.search(pattern, processed_text)
    return res.group() if res else None

  # в случае отсутствия искомого паттерна функция ничего не возвращает
  return None

Функция для поиска транспортных накладных

In [ ]:
def transport_invoice(text):
  # проверка на наличие данных
  if pd.notna(text):
    text = str(text)

    # удаляем id
    text = re.sub(r'\{\d{10}\}', '', text)
    # удаляем пробелы внутри текста
    text = re.sub(r'\s+', '', text)
    # удаляем {}
    text = re.sub(r'[{}]', '', text)
    # удаляем слово "Накладная", учитываем оба регистра
    text = re.sub(r'накладная', '', text, flags=re.I)
    # удаляем слово "ТТН", учитываем оба регистра
    text = re.sub(r'ттн', '', text, flags=re.I)

    # возвращаем данные без пробелов в верхнем регистре
    if text == '':
      return None
    if text:
      return text.upper()

  # если не нашлось данных - возвращаем пустое значение
  return None

Функция для поиска номера трекера

In [ ]:
def tracker_number(text):
  # проверка на наличие данных
  if pd.notna(text):
    text = str(text)

    # убираем лишний текст (" от ДД.ММ.ГГГГ")
    text = re.sub(r'\s+[Оо][Тт].*$', '', text)
    # удаляем id
    text = re.sub(r'\{\d{10}\}', '', text)
    # удаляем слово "ТТН", учитываем оба регистра
    text = re.sub(r'ттн', '', text, flags=re.I)
    # удаляем тире
    text = re.sub('-', '', text)
    # удаляем пробелы
    text = re.sub(r'\s+', '', text)
    # удаляем {}
    text = re.sub(r'[{}]', '', text)

    # если текста фактически не осталось - возвращаем пустое значение
    if text == '':
      return None

    # возвращаем очищенный текст в верхнем регистре
    return text.upper()

  # заглушка
  return None

Функция преобразования приложений к договорам

In [ ]:
def app_nums_processing(text):
  # проверка на наличие данных
  if pd.isna(text):
    return text

  # приводим данные к строковому типу и к верхнему регистру
  text = str(text).upper()

  # убираем лишний текст (" от ДД.ММ.ГГГГ")
  text = re.sub(r'\s+[Оо][Тт].*$', '', text)
  # заменяем все разделители на запятые
  text = re.sub(r'[;\s\.]+', ',', str(text))
  # удаляем лишние запятые и пробелы
  text = re.sub(r',+', ',', text).strip(', ')
  # добавляем пробелы после запятых
  text = re.sub(r',', ', ', text)
  # удаляем возможные пробелы перед запятыми
  text = re.sub(r'\s*,', ',', text)

  # разделяем приложения по запятым и записываем в массив
  applications = re.split(',', text.strip())
  applications = [a for a in applications if a]

  # склеиваем массив обратно в строку
  return ', '.join(applications) if applications else text

Функция преобразования договоров

In [ ]:
def contracts_processing(text):
  # проверка на наличие данных
  if pd.isna(text):
    return text

  # приводим данные к строковому типу и к верхнему регистру
  text = str(text).upper()

  # отдельно приводим формат "Д-р" в нормальный вид
  text = re.sub(r'[Дд]-[Рр]\s+', 'Д_', text)
  # убираем лишний текст (" от ДД.ММ.ГГГГ")
  text = re.sub(r'\s+[Оо][Тт].*$', '', text)

  # ищем пробелы перед типовыми названиями договоров
  pattern = r'\s(?=(Д_|ДП_|ИТС-))'

  # заменяем их на запятые
  text = re.sub(pattern, ',', text)
  # удаляем пробелы внутри договоров
  text = re.sub(' ', '', text)
  # заменяем ; и \ на ,
  text = re.sub(r'[;\\]', ',', text)

  # разделяем договоры по запятым и записываем в массив
  contracts = re.split(',', text.strip())
  contracts = [c for c in contracts if c]

  # склеиваем массив обратно в строку
  return ', '.join(contracts) if contracts else text

Функция извлечения кода с учетом приоритетов столбцов

In [ ]:
def extract_code(text):

  # проверка на наличие данных в ячейке столбца - это необходимо для дальнейшего использования в лямбда-функции
  if pd.notna(text):
    text = str(text)

    # поиск кода в тексте - извлекается группа из одной или более цифры подряд из скобок {}
    res = re.search(r'\{(\d+)\}', text)
    if res:
      # в случае нахождения совпадений возвращается первая извлеченная из скобок группа цифр
      return res.group(1)

    # поиск кода в тексте - извлекается группа из ровно 10 цифр подряд без скобок {}
    res = re.search(r'\b(\d{10})\b', text)
    if res:
      # в случае нахождения совпадений возвращается первая извлеченная из скобок группа цифр
      return res.group(1)

  # в случае отсутствия искомого паттерна функция ничего не возвращает
  return None

Функция статуса проводки СФ

In [ ]:
def colors(row):
  if row['Статус документа счета'] == 'Проведено':
    res = 'Зеленый'
  elif row['Статус документа счета'] == 'Предварительно зарегистрировано' and pd.isna(row['Дата поступл.ТМЦ']) == True:
    res = 'Красный'
  else:
    res = 'Желтый'
  return res

Функция определения категории оприходования СФ

In [ ]:
def reg(df, sap_519, sap_521):

  # разбираем полученные списки СФ на отдельные СФ
  res = df[['Номер документа (сч.ф)']].explode('Номер документа (сч.ф)')

  # удаляем строки без СФ
  res = res[res['Номер документа (сч.ф)'].notna()]

  # цепляем выгрузку SAP 519
  res = res.merge(sap_519[[
      'Номер документа (сч.ф)',
      'Поставка',
      'Статус проводки СФ',
      'Документы проверены',
      'Дата создания СчФ',
      'ТС: Дата отправки',
      'Дата ввода',
      'Дата проводки',
      'Дата поступл.ТМЦ'
      ]].drop_duplicates(), how='left', on='Номер документа (сч.ф)')

  # цепляем выгрузку SAP 521
  res = res.merge(sap_521[['№ приходного ордера']].drop_duplicates(), how='left', left_on='Поставка', right_on='№ приходного ордера')

  # прописываем категории для СФ
  choices = ['Не оприходовано', 'Виртуальный склад', 'Физический склад']

  # прописываем условия для определения СФ в конкретную категорию
  conditions = [
      (res['Статус проводки СФ'] == 'Красный'),
      ~(res['Статус проводки СФ'] == 'Красный') & (res['№ приходного ордера'].notna()),
      ~(res['Статус проводки СФ'] == 'Красный') & (res['№ приходного ордера'].isna())
  ]

  # формируем признак категории для СФ
  res['Оприходование СФ'] = np.select(conditions, choices, default='ПРОВЕРИТЬ!!!')

  # удаляем дубликаты, которые могут появиться после присоединения выгрузок
  res = res.drop_duplicates()

  return res

Функция подсчета СФ

In [ ]:
def analyze_list(lst):

  # обеспечиваем, что lst — список
  if not isinstance(lst, list):
    lst = [lst]

  # по умолчанию ставим метку False на наличие значений и пропусков
  has_nan = False
  has_values = False

  # проверяем наличие и отсутствие пропусков
  for item in lst:
    if item is None or (isinstance(item, float) and np.isnan(item)):
      has_nan = True
    elif item is not None:
      has_values = True

  # найдены и пропуски и значения
  if has_nan and has_values:
    return 'Сч.ф. найдены частично'
  # найдены только пропуски
  elif has_nan and not has_values:
    return 'Не найдено ни одной сч.ф.'
  # найдены только значения
  elif not has_nan:
    return 'Найдены все сч.ф.'

  # заглушка
  return 'Найдены все сч.ф.'

Функция создания ключей с логикой четности

In [ ]:
def generate_keys(contracts, applications, another):

  # приводим данные к строчному типу
  contracts = str(contracts)
  applications = str(applications)

  # убираем пробелы
  contracts = contracts.replace(' ', '')
  applications = applications.replace(' ', '')

  # формируем списки
  contracts = contracts.split(',')
  applications = applications.split(',')

  # это третий аргумент для любого столбца, ячейки которого содержат одно значение
  another_value = str(another).strip()

  # сюда будем складывать найденные ключи
  keys = []

  # правило 1: кол-во договоров = кол-во приложений
  if len(contracts) == len(applications):
    keys.extend([f'{c}_{a}_{another_value}' for c, a in zip(contracts, applications)])

  # правило 2: 1 договор и несколько приложений
  elif len(contracts) == 1 and len(applications) > 1:
    keys.extend([f'{contracts[0]}_{a}_{another_value}' for a in applications])

  # правило 3: несколько договоров и 1 приложение
  elif len(contracts) > 1 and len(applications) == 1:
    keys.extend([f'{c}_{applications[0]}_{another_value}' for c in contracts])

  # правило 4: кол-во договоров > 1, кол-во приложений > 1, кол-ва не равны
  elif len(contracts) > 1 and len(applications) > 1:
    # считаем кол-во приложений на договор с округлением вверх
    multiplicity = math.ceil(len(applications) / len(contracts))
    # определяем с помощью индексов каким договорам соответствуют какие приложения
    # начальный индекс - включительно, конечный - не включительно
    for i, contract in enumerate(contracts):
      start_idx = i * multiplicity
      end_idx = start_idx + multiplicity
      # если это последний договор, то для него берем все приложения с start_idx и до конца списка
      # чтобы не потерять приложения, если общее количество не делится ровно
      if i == len(contracts) - 1:
        for app in applications[start_idx:]:
          keys.append(f'{contract}_{app}_{another_value}')
      # для остальных договоров берем срез приложений с start_idx по end_idx
      else:
        for app in applications[start_idx:end_idx]:
          keys.append(f'{contract}_{app}_{another_value}')

  return keys

Функция создания ключей с логикой разделения через ";"

In [ ]:
def generate_keys2(contracts, applications, another):

  # приводим данные к строчному типу
  contracts = str(contracts)
  applications = str(applications)

  # убираем пробелы
  contracts = contracts.replace(' ', '').replace(';', ',')
  applications = applications.replace(' ', '')

  # формируем списки
  contracts = contracts.split(',')
  applications = applications.split(';')

  # формируем список с группами приложений (каждая группа - список, т.е. получаем список списков)
  app_list = [[str(num.strip()) for num in lst.split(',')] for lst in applications]

  # это третий аргумент для любого столбца, ячейки которого содержат одно значение
  another_value = str(another).strip()

  # сюда будем складывать найденные ключи
  keys = []

  # обрабатываем каждую пару "договор - группа приложений"
  for contract, application_list in zip(contracts, app_list):
    for application in application_list:
      key = f'{contract}_{application}_{another_value}'
      keys.append(key)

  return keys

In [ ]:
# def generate_keys_auto(contracts, applications, another):
#     contracts = str(contracts)
#     contracts = contracts.replace(',', ';')
#     # Автоматический вызов нужной функции
#     if ';' in contracts:
#         return generate_keys2(contracts, applications, another)
#     else:
#         return generate_keys(contracts, applications, another)

Функция соединения данных

In [ ]:
def dfs_merge(df_fca_start, df_fca, df_sap):

  # шаг 1 - ищем соответствие по id
  print('\033[1mШаг первый:\033[0m')

  # копируем исходные реестры во избежание изменений в них
  df_step_one = df_fca.copy()
  sap_exp_one = df_sap.copy()

  # комбинация ключа Д + П + id, попутно форматирование
  df_step_one.loc[:, 'Ключ'] = df_step_one.apply(
      lambda row: generate_keys(
          contracts_processing(row['Договор']),
          app_nums_processing(row['Номер приложения']),
          row['Код']
          ),
      axis=1
      )

  # растаскиваем по строкам
  df_step_one = df_step_one.explode('Ключ').reset_index(drop=True)

  # добавляем номер ключа
  df_step_one['Номер ключа'] = df_step_one.groupby('Индекс перевозки').cumcount() + 1

  # добавляем возможность фильтрации ключей
  df_step_one['Фильтр'] = df_step_one['Индекс перевозки'].astype(str) + '_' + df_step_one['Номер ключа'].astype(str)

  # считаем для каждой строки кол-во раз, которое встречается ключ по ней в таблице
  df_step_one['Маркер Y'] = df_step_one['Ключ'].map(df_step_one['Ключ'].value_counts())

  # удаляем лишние поля и затем дубликаты
  df_step_one = df_step_one[['Индекс перевозки', 'Ключ', 'Номер ключа', 'Фильтр', 'Маркер Y']]
  df_step_one = df_step_one.drop_duplicates()

  # комбинация ключа Д + П + id, попутно форматирование
  sap_exp_one.loc[:, 'Ключ'] = sap_exp_one.apply(
      lambda row: [
          key for val in [extract_code(row[col]) for col in ['ТС: №ТранспДок', 'ТС: №ТранспСр-ва',
                                                             'ТС: №ЕдОбр']]
          for key in generate_keys(
              contracts_processing(row['ЮрНомер договора на поставку']),
              app_nums_processing(row['№ приложения к договору']),
              val
          )
      ],
      axis=1
  )

  # растаскиваем по строкам
  sap_exp_one = sap_exp_one.explode('Ключ')

  # удаляем лишние поля и затем дубликаты
  sap_exp_one = sap_exp_one[['Ключ', 'Номер документа (сч.ф)', 'Дата проводки']]
  sap_exp_one = sap_exp_one.drop_duplicates()

  # соединяем по ключам итоговый на первом шаге датафрейм
  df_step_one = df_step_one.merge(sap_exp_one, how='left', on='Ключ')

  # сч.ф., которые найти не получилось, будем искать на дальнейших итерациях
  df_step_one = df_step_one[~df_step_one['Номер документа (сч.ф)'].isna()]

  # удаляем ключи с Nan на конце
  df_step_one = df_step_one[~df_step_one['Ключ'].str.lower().str.endswith('none')]

  # создаем список отработанных значений-фильтров
  # это значит, что такие индексы ключа для каждой строки больше не будут проверяться, т.к. была найдена сч.ф.
  filter = list(df_step_one['Фильтр'])

  print('Длина полученного датафрейма, ключ - id:', len(df_step_one))

  # кол-во задействованных на шаге индексов
  first_step_len = df_step_one['Индекс перевозки'].nunique()
  # кол-во задействованных накопительным итогом индексов
  sum_len = first_step_len

  # тип соединения
  df_step_one['Тип соединения'] = 'по договору + приложению + id'

  # кол-во найденных на первой итерации уникальных сч.ф.
  num_one = df_step_one['Номер документа (сч.ф)'].nunique()

  print('Кол-во использованных индексов:', first_step_len)
  print('Кол-во найденных сч.ф.:', num_one)
  print('Кол-во строк, которые осталось найти:', len(fca)-sum_len)
  print('Кол-во найденных строк накопительным итогом:', sum_len)
  print('% нахождения накопительным итогом:', round(sum_len / len(fca) * 100, 2))
  print('\033[1m----------------------------------------\033[0m')

  # шаг 2 - ищем соответствие по договору + приложению + ТТН
  print('\033[1mШаг второй:\033[0m')

  # копируем исходные реестры во избежание изменений в них
  df_step_two = df_fca.copy()
  sap_exp_two = df_sap.copy()

  # форматируем датафрейм FCA
  df_step_two.loc[:, 'Ключ'] = df_step_two.apply(
      lambda row: generate_keys(
          contracts_processing(row['Договор']),
          app_nums_processing(row['Номер приложения']),
          transport_invoice(row['№ТН'])
          ),
      axis=1
      )

  # растаскиваем по строкам
  df_step_two = df_step_two.explode('Ключ').reset_index(drop=True)

  # добавляем номер ключа
  df_step_two['Номер ключа'] = df_step_two.groupby('Индекс перевозки').cumcount() + 1

  # добавляем возможность фильтрации ключей
  df_step_two['Фильтр'] = df_step_two['Индекс перевозки'].astype(str) + '_' + df_step_two['Номер ключа'].astype(str)

  # считаем для каждой строки кол-во раз, которое встречается ключ по ней в таблице
  df_step_two['Маркер Y'] = df_step_two['Ключ'].map(df_step_two['Ключ'].value_counts())

  # удаляем лишние поля и затем дубликаты
  df_step_two = df_step_two[['Индекс перевозки', 'Ключ', 'Номер ключа', 'Фильтр', 'Маркер Y']]
  df_step_two = df_step_two.drop_duplicates()

  # форматируем копию датафрейма SAP, созданную для второго шага
  sap_exp_two.loc[:, 'Ключ'] = sap_exp_two.apply(
      lambda row: [
          key for val in [transport_invoice(row[col]) for col in ['ТС: №ТранспДок', 'ТС: №ТранспСр-ва']]
          for key in generate_keys(
              contracts_processing(row['ЮрНомер договора на поставку']),
              app_nums_processing(row['№ приложения к договору']),
              val
          )
      ],
      axis=1
  )

  # растаскиваем по строкам
  sap_exp_two = sap_exp_two.explode('Ключ')

  # удаляем лишние поля и затем дубликаты
  sap_exp_two = sap_exp_two[['Ключ', 'Номер документа (сч.ф)', 'Дата проводки']]
  sap_exp_two = sap_exp_two.drop_duplicates()

  # соединяем по ключам итоговый на втором шаге датафрейм
  df_step_two = df_step_two.merge(sap_exp_two, how='left', on='Ключ')

  # сч.ф., которые найти не получилось, будем искать на дальнейших итерациях
  df_step_two = df_step_two[~df_step_two['Номер документа (сч.ф)'].isna()]

  # удаляем ключи с Nan на конце
  df_step_two = df_step_two[~df_step_two['Ключ'].str.lower().str.endswith('none')]

  # выводим индексы с найденными ключами для строки из поиска
  df_step_two = df_step_two[~df_step_two['Фильтр'].isin(filter)]

  # дополняем список отработанных значений-фильтров
  filter += list(df_step_two['Фильтр'])

  print('Длина полученного датафрейма, ключ - договор + приложение + ТТН:', len(df_step_two))

  # кол-во задействованных на шаге индексов
  second_step_len = df_step_two['Индекс перевозки'].nunique()
  # кол-во задействованных накопительным итогом индексов
  sum_len += second_step_len

  # тип соединения
  df_step_two['Тип соединения'] = 'по договору + приложению + ТТН'

  # кол-во найденных на первой итерации уникальных сч.ф.
  num_two = df_step_two['Номер документа (сч.ф)'].nunique()

  print('Кол-во использованных индексов:', second_step_len)
  print('Кол-во найденных сч.ф.:', num_two)
  print('Кол-во строк, которые осталось найти:', len(fca)-sum_len)
  print('Кол-во найденных строк накопительным итогом:', sum_len)
  print('% нахождения накопительным итогом:', round(sum_len / len(fca) * 100, 2))
  print('\033[1m----------------------------------------\033[0m')

  # шаг 3 - ищем соответствие по договору + приложению + номер трекера
  print('\033[1mШаг третий:\033[0m')

  # копируем исходные реестры во избежание изменений в них
  df_step_three = df_fca.copy()
  sap_exp_three = df_sap.copy()

  # форматируем датафрейм FCA
  df_step_three = df_step_three.copy()
  df_step_three.loc[:, 'Ключ'] = df_step_three.apply(
      lambda row: generate_keys(
          contracts_processing(row['Договор']),
          app_nums_processing(row['Номер приложения']),
          tracker_number(row['Номер трекера'])
          ),
      axis=1
      )

  # растаскиваем по строкам
  df_step_three = df_step_three.explode('Ключ').reset_index(drop=True)

  # добавляем номер ключа
  df_step_three['Номер ключа'] = df_step_three.groupby('Индекс перевозки').cumcount() + 1

  # добавляем возможность фильтрации ключей
  df_step_three['Фильтр'] = df_step_three['Индекс перевозки'].astype(str) + '_' + df_step_three['Номер ключа'].astype(str)

  # считаем для каждой строки кол-во раз, которое встречается ключ по ней в таблице
  df_step_three['Маркер Y'] = df_step_three['Ключ'].map(df_step_three['Ключ'].value_counts())

  # удаляем лишние поля и затем дубликаты
  df_step_three = df_step_three[['Индекс перевозки', 'Ключ', 'Номер ключа', 'Фильтр', 'Маркер Y']]
  df_step_three = df_step_three.drop_duplicates()

  # форматируем копию датафрейма SAP, созданную для третьего шага
  sap_exp_three.loc[:, 'Ключ'] = sap_exp_three.apply(
      lambda row: [
          key for val in [tracker_number(row[col]) for col in ['ТС: №ТранспСр-ва', 'ТС: №ТранспДок', 'ТС: №ЕдОбр']]
          for key in generate_keys(
              contracts_processing(row['ЮрНомер договора на поставку']),
              app_nums_processing(row['№ приложения к договору']),
              val
          )
      ],
      axis=1
  )

  # растаскиваем по строкам
  sap_exp_three = sap_exp_three.explode('Ключ')

  # удаляем лишние поля и затем дубликаты
  sap_exp_three = sap_exp_three[['Ключ', 'Номер документа (сч.ф)', 'Дата проводки']]
  sap_exp_three = sap_exp_three.drop_duplicates()

  # соединяем по ключам итоговый на третьем шаге датафрейм
  df_step_three = df_step_three.merge(sap_exp_three, how='left', on='Ключ')

  # сч.ф., которые найти не получилось, будем искать на дальнейших итерациях
  df_step_three = df_step_three[~df_step_three['Номер документа (сч.ф)'].isna()]

  # # удаляем ключи с Nan на конце
  df_step_three = df_step_three[~df_step_three['Ключ'].str.lower().str.endswith('none')]

  # выводим индексы с найденными ключами для строки из поиска
  df_step_three = df_step_three[~df_step_three['Фильтр'].isin(filter)]

  # дополняем список отработанных значений-фильтров
  filter += list(df_step_three['Фильтр'])

  print('Длина полученного датафрейма, ключ - договор + приложение + номер трекера:', len(df_step_three))

  # кол-во задействованных на шаге индексов
  third_step_len = df_step_three['Индекс перевозки'].nunique()
  # кол-во задействованных накопительным итогом индексов
  sum_len += third_step_len

  # тип соединения
  df_step_three['Тип соединения'] = 'по договору + приложению + номеру трекера'

  # кол-во найденных на первой итерации уникальных сч.ф.
  num_three = df_step_three['Номер документа (сч.ф)'].nunique()

  print('Кол-во использованных индексов:', third_step_len)
  print('Кол-во найденных сч.ф.:', num_three)
  print('Кол-во строк, которые осталось найти:', len(fca)-sum_len)
  print('Кол-во найденных строк накопительным итогом:', sum_len)
  print('% нахождения накопительным итогом:', round(sum_len / len(fca) * 100, 2))
  print('\033[1m----------------------------------------\033[0m')

  # шаг 4 - ищем соответствие по договору + приложению + номер ТС
  print('\033[1mШаг четвертый:\033[0m')

  # копируем исходные реестры во избежание изменений в них
  df_step_four = df_fca.copy()
  sap_exp_four = df_sap.copy()

  # форматируем датафрейм FCA
  df_step_four = df_step_four.copy()
  df_step_four.loc[:, 'Ключ'] = df_step_four.apply(
      lambda row: generate_keys(
          contracts_processing(row['Договор']),
          app_nums_processing(row['Номер приложения']),
          transport_number(row['Номер ТС / накладной'])
          ),
      axis=1
      )

  # растаскиваем по строкам
  df_step_four = df_step_four.explode('Ключ').reset_index(drop=True)

  # добавляем номер ключа
  df_step_four['Номер ключа'] = df_step_four.groupby('Индекс перевозки').cumcount() + 1

  # добавляем возможность фильтрации ключей
  df_step_four['Фильтр'] = df_step_four['Индекс перевозки'].astype(str) + '_' + df_step_four['Номер ключа'].astype(str)

  # считаем для каждой строки кол-во раз, которое встречается ключ по ней в таблице
  df_step_four['Маркер Y'] = df_step_four['Ключ'].map(df_step_four['Ключ'].value_counts())

  # удаляем лишние поля и затем дубликаты
  df_step_four = df_step_four[['Индекс перевозки', 'Ключ', 'Номер ключа', 'Фильтр', 'Маркер Y']]
  df_step_four = df_step_four.drop_duplicates()

  # форматируем копию датафрейма SAP, созданную для четвертого шага
  sap_exp_four.loc[:, 'Ключ'] = sap_exp_four.apply(
      lambda row: [
          key for val in [transport_number(row[col]) for col in ['ТС: №ТранспСр-ва', 'ТС: №ТранспДок']]
          for key in generate_keys(
              contracts_processing(row['ЮрНомер договора на поставку']),
              app_nums_processing(row['№ приложения к договору']),
              val
          )
      ],
      axis=1
      )

  # растаскиваем по строкам
  sap_exp_four = sap_exp_four.explode('Ключ')

  # удаляем лишние поля и затем дубликаты
  sap_exp_four = sap_exp_four[['Ключ', 'Номер документа (сч.ф)', 'Дата проводки']]
  sap_exp_four = sap_exp_four.drop_duplicates()

  # соединяем по ключам итоговый на четвертом шаге датафрейм
  df_step_four = df_step_four.merge(sap_exp_four, how='left', on='Ключ')

  # сч.ф., которые найти не получилось, будем искать на дальнейших итерациях
  df_step_four = df_step_four[~df_step_four['Номер документа (сч.ф)'].isna()]

  # # удаляем ключи с Nan на конце
  df_step_four = df_step_four[~df_step_four['Ключ'].str.lower().str.endswith('none')]

  # выводим индексы с найденными ключами для строки из поиска
  df_step_four = df_step_four[~df_step_four['Фильтр'].isin(filter)]

  # дополняем список отработанных значений-фильтров
  filter += list(df_step_four['Фильтр'])

  print('Длина полученного датафрейма, ключ - договор + приложение + номер ТС:', len(df_step_four))

  # кол-во задействованных на шаге индексов
  fourth_step_len = df_step_four['Индекс перевозки'].nunique()
  # кол-во задействованных накопительным итогом индексов
  sum_len += fourth_step_len

  # тип соединения
  df_step_four['Тип соединения'] = 'по договору + приложению + номеру ТС'

  # кол-во найденных на четвертой итерации уникальных сч.ф.
  num_four = df_step_four['Номер документа (сч.ф)'].nunique()

  print('Кол-во использованных индексов:', fourth_step_len)
  print('Кол-во найденных сч.ф.:', num_four)
  print('Кол-во строк, которые осталось найти:', len(fca)-sum_len)
  print('Кол-во найденных строк накопительным итогом:', sum_len)
  print('% нахождения накопительным итогом:', round(sum_len / len(fca) * 100, 2))
  print('\033[1m----------------------------------------\033[0m')

  # шаг 5 - ищем соответствие по договору + приложению + дата отправки ТС
  print('\033[1mШаг пятый:\033[0m')

  # копируем исходные реестры во избежание изменений в них
  df_step_five = df_fca.copy()
  sap_exp_five = df_sap.copy()

  # форматируем датафрейм FCA
  df_step_five = df_step_five.copy()
  df_step_five.loc[:, 'Ключ'] = df_step_five.apply(
      lambda row: generate_keys(
          contracts_processing(row['Договор']),
          app_nums_processing(row['Номер приложения']),
          row['Дата отгрузки ФАКТ']
          ),
      axis=1
      )

  # растаскиваем по строкам
  df_step_five = df_step_five.explode('Ключ').reset_index(drop=True)

  # добавляем номер ключа
  df_step_five['Номер ключа'] = df_step_five.groupby('Индекс перевозки').cumcount() + 1

  # добавляем возможность фильтрации ключей
  df_step_five['Фильтр'] = df_step_five['Индекс перевозки'].astype(str) + '_' + df_step_five['Номер ключа'].astype(str)

  # считаем для каждой строки кол-во раз, которое встречается ключ по ней в таблице
  df_step_five['Маркер Y'] = df_step_five['Ключ'].map(df_step_five['Ключ'].value_counts())

  # удаляем лишние поля и затем дубликаты
  df_step_five = df_step_five[['Индекс перевозки', 'Ключ', 'Номер ключа', 'Фильтр', 'Маркер Y']]
  df_step_five = df_step_five.drop_duplicates()

  # форматируем копию датафрейма SAP, созданную для пятого шага
  sap_exp_five.loc[:, 'Ключ'] = sap_exp_five.apply(
      lambda row: generate_keys(
          contracts_processing(row['ЮрНомер договора на поставку']),
          app_nums_processing(row['№ приложения к договору']),
          row['ТС: Дата отправки']
          ),
      axis=1
      )

  # растаскиваем по строкам
  sap_exp_five = sap_exp_five.explode('Ключ')

  # удаляем лишние поля и затем дубликаты
  sap_exp_five = sap_exp_five[['Ключ', 'Номер документа (сч.ф)', 'Дата проводки']]
  sap_exp_five = sap_exp_five.drop_duplicates()

  # соединяем по ключам итоговый на пятом шаге датафрейм
  df_step_five = df_step_five.merge(sap_exp_five, how='left', on='Ключ')

  # удаляем ключи с Nan на конце
  df_step_five = df_step_five[~df_step_five['Ключ'].str.lower().str.endswith('none')]

  # выводим индексы с найденными ключами для строки из поиска
  df_step_five = df_step_five[~df_step_five['Фильтр'].isin(filter)]

  # дополняем список отработанных значений-фильтров
  filter += list(df_step_five['Фильтр'])

  print('Длина полученного датафрейма, ключ - договор + приложение + дата отправки ТС:', len(df_step_five))

  # кол-во задействованных на шаге индексов
  fifth_step_len = df_step_five.loc[df_step_five['Номер документа (сч.ф)'].notna(), 'Индекс перевозки'].nunique()
  # кол-во задействованных накопительным итогом индексов
  sum_len += fifth_step_len

  # тип соединения
  df_step_five['Тип соединения'] = 'по договору + приложению + дате отправки ТС'

  # кол-во найденных на пятой итерации уникальных сч.ф.
  num_five = df_step_five['Номер документа (сч.ф)'].nunique()

  print('Кол-во использованных индексов:', fifth_step_len)
  print('Кол-во найденных сч.ф.:', num_five)
  print('Кол-во строк, которые осталось найти:', len(fca)-sum_len)
  print('Кол-во найденных строк накопительным итогом:', sum_len)
  print('% нахождения накопительным итогом:', round(sum_len / len(fca) * 100, 2))
  print('\033[1m----------------------------------------\033[0m')

  print('\033[1mИТОГО:\033[0m')

  # соединяем найденные на предыдущих шагах датафреймы
  res = pd.concat([df_step_one, df_step_two, df_step_three, df_step_four, df_step_five], axis=0, ignore_index=True)

  # сортируем по индексам строк и индексам ключей в строках для корректного порядке при нахождении сч.ф.
  res = res.sort_values(by=['Индекс перевозки', 'Номер ключа'])

  # смотрим по каким ключам сколько соединилось строк
  print(
      res.groupby('Тип соединения')['Номер документа (сч.ф)']
       .size()
       .reset_index(name='Кол-во строк')
       .sort_values(by='Кол-во строк', ascending=False)
       .reset_index(drop=True)
       )


  # создаем таблицу с индексами и сч.ф., которую будем цеплять к исходному реестру FCA, попутно меняет float на int
  res = res.groupby(['Индекс перевозки', 'Маркер Y'], sort=False).agg({
    'Номер документа (сч.ф)': lambda x: [int(val) if pd.notna(val) else val for val in x],
    'Дата проводки': 'max'
    }).reset_index()

  # окончательно схлопываем строки (из наличия маркера Y некоторые строки задублировались)
  res = res.groupby('Индекс перевозки').agg({
    'Маркер Y': 'sum',  # сумма по повтору
    'Номер документа (сч.ф)': 'sum',  # объединение списков
    'Дата проводки': 'max' # дата самой поздней СФ для расчета месяца оприходования поставки
    }).reset_index()

  # переименовываем столбец последней проводки
  res = res.rename(columns={'Дата проводки': 'Дата последней проводки'})

  # цепляем к реестру FCA найденные сч.ф.
  result = df_fca_start.merge(res, how='left', on='Индекс перевозки')

  print('\033[1m----------------------------------------\033[0m')

  # общее кол-во найденных сч.ф.
  invoices = res.explode('Номер документа (сч.ф)')
  n = invoices['Номер документа (сч.ф)'].nunique()
  print(f'\033[1mОбщее кол-во найденных сч.ф.:\033[0m {n}')
  print(f'\033[1m% найденных строк\033[0m {round(sum_len / len(fca) * 100, 2)}')

  return result

# Загрузка и предобработка данных

Загрузка данных

In [ ]:
# загрузка данных
fca = pd.read_excel('Реестр май FCA АТС 2025.xlsx', usecols=[
    'Код',
    'Дата отгрузки ФАКТ',
    'Номер приложения',
    'Договор',
    'Номер ТС / накладной',
    'Номер трекера',
    'Ставка перевозчика с НДС, руб',
    '№ТН'
    ])

sap_519 = pd.read_excel('май 519.XLSX', usecols = [
# sap_519 = pd.read_excel('EXPORT 519 FULL.XLSX', usecols = [
    'ЮрНомер договора на поставку',
    '№ приложения к договору',
    'Номер документа (сч.ф)',
    'ТС: №ТранспСр-ва',
    'ТС: №ТранспДок',
    'ТС: Дата отправки',
    'ТС: №ЕдОбр',
    'Поставка',
    'Статус документа счета',
    'Дата поступл.ТМЦ',
    'Документы проверены',
    'Дата создания СчФ',
    'Дата ввода',
    'Дата проводки'
    ])

sap_519_ed = pd.read_excel('май 519 ЭД.XLSX', usecols=[
    'ЮрНомер договора на поставку',
    '№ приложения к договору',
    'Номер документа (сч.ф)',
    'ТС: №ТранспСр-ва',
    'ТС: №ТранспДок',
    'ТС: Дата отправки',
    'ТС: №ЕдОбр',
    'Счет-фактура',
    'Поставка',
    'Статус документа счета',
    'Дата поступл.ТМЦ',
    'Документы проверены',
    'Дата создания СчФ',
    'Дата ввода',
    'Дата проводки'
])

sap_521 = pd.read_excel('май 521.XLSX')

logs = pd.read_excel('Логи_Май.xlsx')

print('Длина FCA:', len(fca))
print('Длина SAP 519:', len(sap_519))
print('Длина SAP 519 ЭД:', len(sap_519_ed))

Длина FCA: 1932
Длина SAP 519: 3229
Длина SAP 519 ЭД: 8184


Выбор периода для расчета

In [ ]:
# строго
period = input('Выберите период формирования файлов (введите МЕСЯЦ или ГОД): ').strip().lower()

# контроль
if period == 'год':
  print('Выбранный период - ГОД')
elif period == 'месяц':
  print('ВЫбранный период - Месяц')
else:
  raise ValueError(f'❌ Неизвестный период: {period}. Допустимые значения: МЕСЯЦ, ГОД')

Выберите период формирования файлов (введите МЕСЯЦ или ГОД): год
Выбранный период - ГОД


In [ ]:
# выбор месяца (при необходимости)
if period == 'месяц':
  month = input('Выберите МЕСЯЦ: ').strip().lower()

  if month == 'январь':
    mm = '02'
  elif month == 'февраль':
    mm = '03'
  elif month == 'март':
    mm = '04'
  elif month == 'апрель':
    mm = '05'
  elif month == 'май':
    mm = '06'
  elif month == 'июнь':
    mm = '07'
  elif month == 'июль':
    mm = '08'
  elif month == 'август':
    mm = '09'
  elif month == 'сентябрь':
    mm = '10'
  elif month == 'октябрь':
    mm = '11'
  elif month == 'ноябрь':
    mm = '12'
  elif month == 'декабрь':
    mm = '01'
  else:
    raise ValueError(f'❌ Ошибка при выборе месяца: {month}. Необходимо полностью и корректно ввести месяц')
else:
  pass

In [ ]:
if period == 'месяц':

  # исключаем данные за следующий месяц
  fca = fca[~(fca['Код'].str[2:4] == mm)]
  print('Длина после перепроверки FCA:', len(fca))

else:
  pass

In [ ]:
# добавляем индекс каждой строке реестра FCA
fca.insert(0, 'Индекс перевозки', np.arange(0, len(fca)))

In [ ]:
# поля, которые блок FCA должен заполнить данными
cols_revision = [
    'Код',
    'Договор',
    'Номер приложения',
    'Дата отгрузки ФАКТ',
    'Ставка перевозчика с НДС, руб'
    ]

# таблица, которая будет направлена блоку FCA на дозаполнение
fca_revision = fca[fca[cols_revision].isnull().any(axis=1)]

Преобразование выгрузки SAP 519

In [ ]:
# корректировка сч.ф. в выгрузке SAP 519 ЭД
mask = sap_519_ed['Номер документа (сч.ф)'] == 1
sap_519_ed.loc[mask, 'Номер документа (сч.ф)'] = sap_519_ed.loc[mask, 'Счет-фактура']

# данный столбец более не нужен, удаляем его
sap_519_ed = sap_519_ed.drop('Счет-фактура', axis=1)

# приводим в единый вид выгрузки SAP 519 и SAP 519 ЭД, должен соблюдаться порядок столбцов
sap_519 = sap_519[[
    'ЮрНомер договора на поставку',
    '№ приложения к договору',
    'ТС: Дата отправки',
    'Номер документа (сч.ф)',
    'ТС: №ЕдОбр',
    'ТС: №ТранспДок',
    'ТС: №ТранспСр-ва',
    'Поставка',
    'Статус документа счета',
    'Дата поступл.ТМЦ',
    'Документы проверены',
    'Дата создания СчФ',
    'Дата ввода',
    'Дата проводки'
    ]]

sap_519_ed = sap_519_ed[[
    'ЮрНомер договора на поставку',
    '№ приложения к договору',
    'ТС: Дата отправки',
    'Номер документа (сч.ф)',
    'ТС: №ЕдОбр',
    'ТС: №ТранспДок',
    'ТС: №ТранспСр-ва',
    'Поставка',
    'Статус документа счета',
    'Дата поступл.ТМЦ',
    'Документы проверены',
    'Дата создания СчФ',
    'Дата ввода',
    'Дата проводки'
    ]]

# соединяем реестры SAP по вертикали
sap_519_full = pd.concat([sap_519, sap_519_ed], axis=0, ignore_index=True)

# избавляемся от всех строк SAP 519, где нет СФ
sap_519_full = sap_519_full[sap_519_full['Номер документа (сч.ф)'].notna()]

# добавляем статус проводки
sap_519_full['Статус проводки СФ'] = sap_519_full.apply(colors, axis=1)

# приводим столбец статуса документов в единый регистр
sap_519_full['Документы проверены'] = sap_519_full['Документы проверены'].str.lower()

# меняем тип данных на текстовый
# это дополнительная мера, т.к. тип данных в любом случае меняется внутри функций
cols_fca_str = ['Номер ТС / накладной', 'Номер приложения', 'Номер трекера', '№ТН']
for col in cols_fca_str:
  fca[col] = fca[col].where(fca[col].isna(), fca[col].astype('str'))

cols_sap_str = [
    'ТС: №ТранспСр-ва',
    'ТС: №ТранспДок',
    'ТС: №ЕдОбр',
    'ЮрНомер договора на поставку',
    '№ приложения к договору',
    ]
for col in cols_sap_str:
  sap_519_full[col] = sap_519_full[col].where(sap_519_full[col].isna(), sap_519_full[col].astype('str'))

# на всякий случай присваиваем СФ тип данных int
sap_519_full['Номер документа (сч.ф)'] = sap_519_full['Номер документа (сч.ф)'].astype(int)

# будущий ключ SAP 519 приводим к целочисленному типу данных
sap_519_full['Поставка'] = pd.to_numeric(sap_519_full['Поставка'], errors='coerce')
sap_519_full['Поставка'] = sap_519_full['Поставка'].astype('Int64')

# избавляемся от лишних скобок в поле id
fca['Код'] = fca['Код'].str.replace(r'[{}]', '', regex=True)

# преобразуем поля из текствого формата в списки
col_list = ['ТС: №ТранспСр-ва', 'ТС: №ТранспДок', 'ТС: №ЕдОбр']
sap_519_full[col_list] = sap_519_full[col_list].applymap(
    lambda x: [item.strip() for item in x.split(',')] if pd.notna(x) else []
)

# растаскиваем по строкам
sap_519_exp = sap_519_full.explode('ТС: №ТранспСр-ва').explode('ТС: №ТранспДок').explode('ТС: №ЕдОбр')

# чистим столбцы
for col in col_list:
    mask = sap_519_exp[col].str.lower() == 'нет данных'
    sap_519_exp.loc[mask, col] = None

/tmp/ipython-input-205175834.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  sap_519_ed.loc[mask, 'Номер документа (сч.ф)'] = sap_519_ed.loc[mask, 'Счет-фактура']
/tmp/ipython-input-205175834.py:83: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  sap_519_full[col_list] = sap_519_full[col_list].applymap(


Преобразование выгрузки SAP 521

In [ ]:
# избавляемся от строк без необходимого в дальнейшем ключа
sap_521 = sap_521[sap_521['№ приходного ордера'].notna()]

# будущий ключ SAP 521 приводим к типу данных float
sap_521['№ приходного ордера'] = pd.to_numeric(sap_521['№ приходного ордера'], errors='coerce')

# Прямое преобразование в int (без промежуточного float)
sap_521['№ приходного ордера'] = sap_521['№ приходного ордера'].astype('Int64')

Преобразование логов

In [ ]:
# во избежание присоединения лишних строк из логов выделяем самые поздние для каждой СФ
logs = logs.groupby('№ докум.')['ДатаИзм'].max().reset_index()

In [ ]:
# исключаем часть реестра FCA, отправленную на доработку
fca_clean = fca[~fca['Индекс перевозки'].isin(fca_revision['Индекс перевозки'])]

In [ ]:
# соединяем реестры
df_res = dfs_merge(fca, fca_clean, sap_519_exp)

Шаг первый:
Длина полученного датафрейма, ключ - id: 556
Кол-во использованных индексов: 470
Кол-во найденных сч.ф.: 548
Кол-во строк, которые осталось найти: 1462
Кол-во найденных строк накопительным итогом: 470
% нахождения накопительным итогом: 24.33
----------------------------------------
Шаг второй:
Длина полученного датафрейма, ключ - договор + приложение + ТТН: 21
Кол-во использованных индексов: 20
Кол-во найденных сч.ф.: 21
Кол-во строк, которые осталось найти: 1442
Кол-во найденных строк накопительным итогом: 490
% нахождения накопительным итогом: 25.36
----------------------------------------
Шаг третий:
Длина полученного датафрейма, ключ - договор + приложение + номер трекера: 91
Кол-во использованных индексов: 57
Кол-во найденных сч.ф.: 91
Кол-во строк, которые осталось найти: 1385
Кол-во найденных строк накопительным итогом: 547
% нахождения накопительным итогом: 28.31
----------------------------------------
Шаг четвертый:
Длина полученного датафрейма, ключ - договор + п

# Преобразование полученных данных

In [ ]:
# маркер Y

# это метка для случая, когда M одинаковых ключей и N сч.ф.
# т.е. проблема, что нельзя установить точное соответствие между сч.ф. и ключом
# в идеале такое должно быть только на итерациях 4 и 5 - по номеру ТС и дате отправки
df_res['Маркер Y'] = np.where((df_res['Маркер Y'] == 1) | (df_res['Маркер Y'].isna()), 'N', 'Y')

In [ ]:
# маркер X

# это проверка того, найдены ли все сч.ф. или нет
df_res['Маркер X'] = df_res['Номер документа (сч.ф)'].apply(analyze_list)

# корректируем маркер Y - не будем помечать им строки без сч.ф.
df_res.loc[df_res['Маркер X'] == 'Не найдено ни одной сч.ф.', 'Маркер Y'] = 'N'

In [ ]:
# найденные в FCA перевозки
df_res['Зарегистрировано'] = np.where(df_res['Маркер X'] == 'Не найдено ни одной сч.ф.', 0, 1)

# не найденные в FCA перевозки
df_res['Не найдено / Не зарегистрировано'] = np.where(df_res['Маркер X'] == 'Не найдено ни одной сч.ф.', 1, 0)

In [ ]:
# соединяем найденные СФ с выгрузками SAP 519 и SAP 521
inv_reg = reg(df_res, sap_519_full, sap_521)

In [ ]:
# оприходована ли поставка (найдены все СФ)
df_res['Оприходование поставки'] = np.where(df_res['Маркер X'] == 'Найдены все сч.ф.', 1, 0)

# указываем статус перевозки в FCA
df_res['Статус перевозки'] = np.where(
    df_res['Не найдено / Не зарегистрировано'] == 1,
    'Не найдено / Не зарегистрировано',
    np.where(df_res['Оприходование поставки'] == 1, 'Оприходовано', 'Не оприходовано')
    )

In [ ]:
# указываем месяц реестра FCA
df_res['Месяц'] = df_res['Дата отгрузки ФАКТ'].dt.month

# словарь соответствия
month_dict = {
    1: 'Январь', 2: 'Февраль', 3: 'Март', 4: 'Апрель',
    5: 'Май', 6: 'Июнь', 7: 'Июль', 8: 'Август',
    9: 'Сентябрь', 10: 'Октябрь', 11: 'Ноябрь', 12: 'Декабрь'
}

# указываем наименование месяца реестра FCA
df_res['Месяц наименование'] = df_res['Месяц'].map(month_dict)

In [ ]:
# цепляем логи к списку найденных СФ
inv_log = inv_reg.merge(logs, how='left', left_on='Номер документа (сч.ф)', right_on='№ докум.')

In [ ]:
# рассчитываем время проверки документов в днях
inv_log['Время проверки документов'] = np.where(
    inv_log['ДатаИзм'].isna(),
    (inv_log['Дата создания СчФ'] - inv_log['ТС: Дата отправки']).dt.days,
    (inv_log['ДатаИзм'] - inv_log['ТС: Дата отправки']).dt.days
    ).astype(int)

In [ ]:
# рассчитываем время проводки документов в днях
inv_log['Время проводки документов'] = np.where(
    inv_log['ДатаИзм'].isna(),
    (inv_log['Дата ввода'] - inv_log['Дата создания СчФ']).dt.days,
    (inv_log['Дата ввода'] - inv_log['ДатаИзм']).dt.days
    ).astype(int)

In [ ]:
# факт оприходования СФ
inv_log['Оприходование СФ факт'] = np.where(inv_log['Оприходование СФ'].isin(['Физический склад', 'Виртуальный склад']), 1, 0)

In [ ]:
# виртуальный склад (да/нет)
inv_log['Виртуальный склад'] = np.where(inv_log['Оприходование СФ'] == 'Виртуальный склад', 1, 0)

# физический склад (да/нет)
inv_log['Физический склад'] = np.where(inv_log['Оприходование СФ'] == 'Физический склад', 1, 0)

Дополнительно проверяем корректность указания id в реестре FCA

In [ ]:
# побочные датафреймы для отлавливания ошибок
for_errors_df_one = fca_clean.copy()
for_errors_sap_one = sap_519_exp.copy()

# форматируем побочный датафрейм, по которому будем искать разночтения по id
# на данном шаге можно использовать merge по полю "Код"
# но для получения одинакого кол-ва полей у побочных датафреймов лучше использовать функцию
for_errors_df_one['Ключ'] = for_errors_df_one.apply(
    lambda row: generate_keys(
        contracts_processing(row['Код']),
        app_nums_processing(row['Код']),
        row['Код']
        ), axis=1
    )

# растаскиваем по строкам
for_errors_df_one = for_errors_df_one.explode('Ключ')

# форматируем побочный датафрейм, по которому будем искать разночтения по id
for_errors_sap_one['Ключ'] = for_errors_sap_one.apply(
    lambda row: [
        key for val in [extract_code(row[col]) for col in ['ТС: №ТранспДок', 'ТС: №ТранспСр-ва', 'ТС: №ЕдОбр']]
        for key in generate_keys(
            val,
            val,
            val
        )
    ],
    axis=1
    )

# растаскиваем по строкам
for_errors_sap_one = for_errors_sap_one.explode('Ключ')

# соединяем по ключам датафреймы для поиска ошибок
for_errors_df_one = for_errors_df_one.merge(for_errors_sap_one, how='inner', on='Ключ')

# удаляем ключи с Nan на конце
for_errors_df_one = for_errors_df_one[~for_errors_df_one['Ключ'].str.lower().str.endswith('none')]

**Сохраняем результат**

In [ ]:
# создаем в локальной директории excel-файл с финальными данными, откуда его можно будет скачать
df_final = [df_res]
sheet_names = ['Данные']
excel_filename = 'Модель данных.xlsx'
with pd.ExcelWriter(excel_filename) as writer:
  for data, sheet in zip(df_final, sheet_names):
    df = pd.DataFrame(data)
    df.to_excel(writer, sheet_name=sheet, header=True)

Формирование дополнительного реестра на проверку FCA

In [ ]:
# ищем строки с одинаковым id и разными номерами договора
erros_df = for_errors_df_one[
    (for_errors_df_one['Договор'].str.strip() != for_errors_df_one['ЮрНомер договора на поставку'].str.strip()) &
    (~for_errors_df_one['ЮрНомер договора на поставку'].fillna('').str.contains(r'[\;\,\/\\]')) &
    (for_errors_df_one['ЮрНомер договора на поставку'].str.count('Д') < 2) &
    (~for_errors_df_one['Договор'].str.contains(r'[\;\,\/\\]')) &
    (for_errors_df_one['Договор'].str.count('Д') < 2)
    ]

erros_df = erros_df[[
    'Договор',
    'Номер приложения',
    'ЮрНомер договора на поставку',
    '№ приложения к договору'
    ]].drop_duplicates()

# Формирование Excel-файла на доработку FCA

In [ ]:
# формируем файл для возврата на проверку блоку FCA
# первый лист - не до конца заполненные обязательные данные
# второй лист - строки, сцепившиеся по одному id, но с разными номерами договоров и приложениями; это на уточнение

datasets = [fca_revision, erros_df]
sheet_names = ['Нет данных', 'На уточнение']
excel_filename = 'FCA реестр на проверку.xlsx'

with pd.ExcelWriter(excel_filename) as writer:
  for data, sheet in zip(datasets, sheet_names):
    df = pd.DataFrame(data)
    df.to_excel(writer, sheet_name=sheet, header=True, index=False)

# Расчеты для первой страницы дашборда "МТР в пути" (месяц)

**1. Информация о перевозках за соответствующий месяц**

In [ ]:
if period == 'месяц':

  # кол-во найденных перевозок
  fnd_tr = sum(df_res['Зарегистрировано'])

  # кол-во не найденных перевозок
  nt_fnd_tr = sum(df_res['Не найдено / Не зарегистрировано'])

  print(f'Общее кол-во перевозок FCA: {len(fca)} шт.;')
  print(f'Возвращено на доработку: {len(fca_revision)} шт., {round(len(fca_revision) / len(fca) * 100, 2)} %;')
  print('')
  print(f'Зарегистрировано: {fnd_tr} шт., {round(fnd_tr / len(fca) * 100, 2)} %;')
  print(f'Не найдено / Не зарегистрировано: {nt_fnd_tr} шт., {round(nt_fnd_tr / len(fca) * 100, 2)} %;')

else:
  pass

**2. Определение категории оприходования СФ**

In [ ]:
if period == 'месяц':

  # статусы оприходования по СФ
  print(inv_log.groupby('Оприходование СФ')['Номер документа (сч.ф)'].nunique().reset_index(name='Кол-во СФ'))

else:
  pass

**3. Статусы СФ по документам зарегистрированных в SAP поставок**

In [ ]:
if period == 'месяц':

  # фиксируем получение статусов проверки документов для отражения в дашборде
  print(inv_log.groupby('Документы проверены')['Номер документа (сч.ф)'].nunique().reset_index(name='Кол-во СФ'))

else:
  pass

**4. Расчет стоимости перевозок по оприходованным МТР**

In [ ]:
if period == 'месяц':

  # суммарная стоимость перевозок, руб
  print(
      df_res.groupby('Статус перевозки')['Ставка перевозчика с НДС, руб'].sum().apply(
          lambda x: f'{x:,.0f}'.replace(',', ' ')).reset_index(name='Суммарная стоимость перевозок (с НДС), руб'
          )
      )

else:
  pass

**5. Определение категорий времени для СФ с проверенными документами (с момента отправки ТС)**

In [ ]:
if period == 'месяц':

  # прописываем категории для времени проверки документов
  choices_check = ['1.До 5 дней', '2.От 6 до 10 дней', '3.От 11 до 14 дней', '4.Более 14 дней']

  # прописываем условия для определения СФ в конкретную категорию времени проверки документов
  conditions_check = [
      inv_log['Время проверки документов'] <= 5,
      (inv_log['Время проверки документов'] <= 10) & (inv_log['Время проверки документов'] > 5),
      (inv_log['Время проверки документов'] <= 14) & (inv_log['Время проверки документов'] > 10),
      inv_log['Время проверки документов'] > 14
  ]

  # формируем признак категории для СФ
  inv_log['Время проверки документов (группы)'] = np.select(conditions_check, choices_check, default='ПРОВЕРИТЬ!!!')

  print(
      inv_log.groupby('Время проверки документов (группы)')['Номер документа (сч.ф)'].nunique()
  )

else:
  pass

**6. Определение категорий времени для оприходованных СФ (на склад с момента проверки документов)**

In [ ]:
if period == 'месяц':

  # прописываем категории для времени проводки документов
  choices_post = ['1.Без задержек', '2.От 1 до 3 дней', '3.От 4 до 5 дней', '4.От 6 до 10 дней', '5.От 11 до 14 дней', '6.Более 14 дней']

  # прописываем условия для определения СФ в конкретную категорию времени проводки документов
  conditions_post = [
      inv_log['Время проводки документов'] <= 0,
      (inv_log['Время проводки документов'] <= 3) & (inv_log['Время проводки документов'] > 0),
      (inv_log['Время проводки документов'] <= 5) & (inv_log['Время проводки документов'] > 3),
      (inv_log['Время проводки документов'] <= 10) & (inv_log['Время проводки документов'] > 5),
      (inv_log['Время проводки документов'] <= 14) & (inv_log['Время проводки документов'] > 10),
      inv_log['Время проводки документов'] > 14
  ]

  # формируем признак категории для СФ
  inv_log['Время проводки документов (группы)'] = np.select(conditions_post, choices_post, default='ПРОВЕРИТЬ!!!')

  print(
      inv_log.groupby('Время проводки документов (группы)')['Номер документа (сч.ф)'].nunique()
  )

else:
  pass

**Сохранение результатов (месяц)**

In [ ]:
if period == 'месяц':

  datasets = [df_res, inv_log]
  sheet_names = ['FCA мес', 'SAP мес']
  excel_filename = 'Модель данных месяц.xlsx'

  with pd.ExcelWriter(excel_filename) as writer:
    for data, sheet in zip(datasets, sheet_names):
      df = pd.DataFrame(data)
      df.to_excel(writer, sheet_name=sheet, header=True, index=False)

else:
  pass

# Расчеты для второй страницы дашборда "МТР в пути" (год)

**1. Динамика оприходования перевозок после закрытия месяца**

In [ ]:
if period == 'год':

  # категории оприходования
  fca_cat = ['В рамках месяца', 'После закрытия месяца', 'Не оприходовано']

  # условия для категорий оприходования
  fca_cond = [
      (df_res['Оприходование поставки'] == 1) & (
          (df_res['Дата последней проводки'].dt.month - df_res['Месяц'] == 0) | (
              (df_res['Дата последней проводки'].dt.month - df_res['Месяц'] == 1) & (df_res['Дата последней проводки'].dt.day <= 6)
          )
          ),
      (df_res['Оприходование поставки'] == 1) & (
          (
              (df_res['Дата последней проводки'].dt.month - df_res['Месяц'] == 1) & (df_res['Дата последней проводки'].dt.day > 6)
              ) |
              (df_res['Дата последней проводки'].dt.month - df_res['Месяц'] > 1)
              ),
      (df_res['Оприходование поставки'] == 0)
      ]

  # данный столбец нужен для определения своевременности оприходования поставки в рамках года
  df_res['Категория оприходования'] = np.select(fca_cond, fca_cat, default='ПРОВЕРИТЬ!!!')

  # в рамках месяца (да/нет)
  df_res['В рамках месяца'] = np.where(df_res['Категория оприходования'] == 'В рамках месяца', 1, 0)

  # после закрытия месяца (да/нет)
  df_res['После закрытия месяца'] = np.where(df_res['Категория оприходования'] == 'После закрытия месяца', 1, 0)

  print(
      df_res.groupby(['Месяц наименование', 'Категория оприходования'])['Индекс перевозки'].nunique()
  )

else:
  pass

Месяц наименование  Категория оприходования
Май                 В рамках месяца            999
                    Не оприходовано            505
                    После закрытия месяца      428
Name: Индекс перевозки, dtype: int64


**2. Стоимость перевозок по оприходованным МТР**

In [ ]:
if period == 'год':

  # суммарная стоимость перевозок, руб
  print(
      df_res.groupby('Категория оприходования')['Ставка перевозчика с НДС, руб'].sum().apply(
          lambda x: f'{x:,.0f}'.replace(',', ' ')).reset_index(name='Суммарная стоимость перевозок (с НДС), руб'
          )
      )

else:
  pass

  Категория оприходования Суммарная стоимость перевозок (с НДС), руб
0         В рамках месяца                                 97 915 094
1         Не оприходовано                                 74 350 832
2   После закрытия месяца                                 53 734 093


**3. Оприходованные перевозки (на склад с момента отправки ТС)**

In [ ]:
if period == 'год':

  # прописываем категории для времени оприходования перевозки
  choices_post = ['1.До 5 дней', '2.6-10 дней', '3.11-20 дней', '4.21-30 дней', '5.Более 30 дней']

  # прописываем условия для определения СФ в конкретную категорию времени проводки документов
  conditions_post = [
      (df_res['Дата последней проводки'] - df_res['Дата отгрузки ФАКТ']).dt.days <= 5,
      ((df_res['Дата последней проводки'] - df_res['Дата отгрузки ФАКТ']).dt.days <= 10) & ((df_res['Дата последней проводки'] - df_res['Дата отгрузки ФАКТ']).dt.days > 5),
      ((df_res['Дата последней проводки'] - df_res['Дата отгрузки ФАКТ']).dt.days <= 20) & ((df_res['Дата последней проводки'] - df_res['Дата отгрузки ФАКТ']).dt.days > 10),
      ((df_res['Дата последней проводки'] - df_res['Дата отгрузки ФАКТ']).dt.days <= 30) & ((df_res['Дата последней проводки'] - df_res['Дата отгрузки ФАКТ']).dt.days > 20),
      (df_res['Дата последней проводки'] - df_res['Дата отгрузки ФАКТ']).dt.days > 30
  ]

  # формируем признак категории для СФ
  df_res['Оприходование поставки (категории)'] = np.select(conditions_post, choices_post, default='Не оприходовано')

  print(
      df_res.groupby('Оприходование поставки (категории)')['Оприходование поставки'].sum()
  )

else:
  pass

Оприходование поставки (категории)
1.До 5 дней        470
2.6-10 дней        284
3.11-20 дней       303
4.21-30 дней       151
5.Более 30 дней    219
Не оприходовано      0
Name: Оприходование поставки, dtype: int64


**4. Оприходованные СФ (на склад с момента проверки документов)**

In [ ]:
if period == 'год':

  # прописываем категории для времени проводки документов
  choices_post = ['1.До 5 дней', '2.6-10 дней', '3.11-20 дней', '4.21-30 дней', '5.Более 30 дней']

  # прописываем условия для определения СФ в конкретную категорию времени проводки документов
  conditions_post = [
      inv_log['Время проводки документов'] <= 5,
      (inv_log['Время проводки документов'] <= 10) & (inv_log['Время проводки документов'] > 5),
      (inv_log['Время проводки документов'] <= 20) & (inv_log['Время проводки документов'] > 10),
      (inv_log['Время проводки документов'] <= 30) & (inv_log['Время проводки документов'] > 20),
      inv_log['Время проводки документов'] > 30
  ]

  # формируем признак категории для СФ
  inv_log['Время проводки документов (группы)'] = np.select(conditions_post, choices_post, default='Не оприходовано')

  print(
      inv_log[inv_log['Оприходование СФ'] != 'Не оприходовано'].groupby(['Время проводки документов (группы)', 'Оприходование СФ'])['Номер документа (сч.ф)'].nunique()
  )

else:
  pass

Время проводки документов (группы)  Оприходование СФ 
1.До 5 дней                         Виртуальный склад    124
                                    Физический склад     376
2.6-10 дней                         Виртуальный склад     39
                                    Физический склад     286
3.11-20 дней                        Виртуальный склад     23
                                    Физический склад     374
4.21-30 дней                        Виртуальный склад      5
                                    Физический склад     169
5.Более 30 дней                     Физический склад     171
Name: Номер документа (сч.ф), dtype: int64


**Сохранение результатов (год)**

In [ ]:
if period == 'год':

  datasets = [df_res, inv_log]
  sheet_names = ['FCA год', 'SAP год']
  excel_filename = 'Модель данных год.xlsx'

  with pd.ExcelWriter(excel_filename) as writer:
    for data, sheet in zip(datasets, sheet_names):
      df = pd.DataFrame(data)
      df.to_excel(writer, sheet_name=sheet, header=True, index=False)

else:
  pass